In [10]:
import geopandas as gpd
import pandas as pd
from osgeo import gdal

# Chemins des fichiers
input_shp = "../../data/project/FORMATION_VEGETALE.shp"  # Shapefile d'entrée
input_raster = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"  # Raster d'entrée
output_shp = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"  # Shapefile de sortie

# Charger l'emprise du raster
raster = gdal.Open(input_raster)
geo_transform = raster.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * raster.RasterXSize
y_min = y_max + geo_transform[5] * raster.RasterYSize

# Charger le shapefile
gdf = gpd.read_file(input_shp)

# Obtenir le CRS du raster
raster_crs = raster.GetProjection()

# Reprojeter le GeoDataFrame pour qu'il corresponde au CRS du raster
if gdf.crs.to_string() != raster_crs:
    print(f"Le CRS du shapefile ({gdf.crs}) est différent du CRS du raster. Reprojection en cours.")
    gdf = gdf.to_crs(raster_crs)  # Reprojection du shapefile vers le CRS du raster
    print(f"Le shapefile a été reprojeté vers le CRS du raster : {raster_crs}")
else:
    print("Le CRS du shapefile et du raster sont déjà identiques.")

# Découper le shapefile en utilisant les limites du raster
clipped_gdf = gdf.cx[x_min:x_max, y_min:y_max]

# Attributs TFV à traiter et leurs correspondances
mapping = {
    "Forêt fermée d’un autre feuillu pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de châtaignier pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de hêtre pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de chênes décidus purs": ("Chêne", 12, "Chêne", 12),
    "Forêt fermée de robinier pur": ("Robinier", 13, "Robinier", 13),
    "Peuplerai": ("Peupleraie", 14, "Peupleraie", 14),
    "Forêt fermée à mélange de feuillus": (None, None, "Mélange de feuillus", 15),
    "Forêt fermée de feuillus purs en îlots": (None, None, "feuillus en îlots", 16),
    "Forêt fermée d’un autre conifère pur autre que pin": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de mélèze pur": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de sapin ou épicéa": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée à mélange d’autres conifères": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée d’un autre pin pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de pin sylvestre pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée à mélange de pins purs": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de douglas pur": ("Douglas", 23, "Douglas", 23),
    "Forêt fermée de pin laricio ou pin noir pur": ("Pin laricio ou pin noir", 24, "Pin laricio ou pin noir", 24),
    "Forêt fermée de pin maritime pur": ("Pin maritime", 25, "Pin maritime", 25),
    "Forêt fermée à mélange de conifères": (None, None, "Mélange conifères", 26),
    "Forêt fermée de conifères en îlots": (None, None, "Conifères en îlots", 27),
    "Forêt fermée à mélange de conifères prépondérants et feuillus": (None, None, "Mélange de conifères prépondérants et feuillus", 28),
    "Forêt fermée à mélange de feuillus prépondérants et conifères": (None, None, "Mélange de feuillus prépondérants et conifères", 29),
}

# Ajouter les nouveaux champs "Nom" et "Code" (2 fois pour pixel et objet)
def map_tfv(tfv):
    return mapping.get(tfv, (None, None, None, None))  # Toujours retourner 4 éléments

# Vérifiez que la colonne "TFV" existe
if "TFV" in clipped_gdf.columns:
    # Appliquer le mappage et créer les colonnes
    mapped_values = clipped_gdf["TFV"].apply(map_tfv)
    mapped_df = pd.DataFrame(mapped_values.tolist(), columns=["Nom_pixel", "Code_pixel", "Nom_objet", "Code_objet"])
    
    # Ajouter les colonnes mappées au GeoDataFrame
    clipped_gdf = pd.concat([clipped_gdf, mapped_df], axis=1)
    
    print("Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.")
else:
    print("Erreur : La colonne 'TFV' est absente du GeoDataFrame. Impossible de créer les nouvelles colonnes.")

# Sauvegarder le fichier shapefile résultant
clipped_gdf.to_file(output_shp, driver="ESRI Shapefile")

print(f"Fichier shapefile créé : {output_shp}")


Le CRS du shapefile (EPSG:2154) est différent du CRS du raster. Reprojection en cours.
Le shapefile a été reprojeté vers le CRS du raster : PROJCS["WGS 84 / UTM zone 31N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32631"]]
Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.
Fichier shapefile créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp
